In [48]:
# basic
import numpy as np
import pandas as pd

# get data
import pandas_datareader as pdr

# visual
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#requests
import requests
def remove_td(column):
    remove_one = column.split('<')
    remove_two = remove_one[0].split('>')
    return remove_two[1].replace(",", "")

def translate_dataFrame(response):
     # 拆解內容
    table_array = response.split('<table')
    tr_array = table_array[1].split('<tr')
    
    # 拆解td
    data = []
    index = []
    column = []
    for i in range(len(tr_array)):
        td_array = tr_array[i].split('<td')
        if(len(td_array)>1):
            code = remove_td(td_array[1])
            name = remove_td(td_array[2])
            revenue  = remove_td(td_array[3])
            profitRatio = remove_td(td_array[4])
            profitMargin = remove_td(td_array[5])
            preTaxIncomeMargin = remove_td(td_array[6])
            afterTaxIncomeMargin = remove_td(td_array[7])
            if(type(code) == float):
                data.append([code, revenue, profitRatio, profitMargin, preTaxIncomeMargin, afterTaxIncomeMargin])
                index.append(name)
            if( i == 1 ):
                column.append(code)
                column.append(revenue)
                column.append(profitRatio)
                column.append(profitMargin)
                column.append(preTaxIncomeMargin)
                column.append(afterTaxIncomeMargin)
                
    return pd.DataFrame(data=data, index=index, columns=column)

In [57]:
 
url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb06'
form_data = {
'encodeURIComponent':1,
'step':1,
'firstin':1,
'off':1,
'TYPEK':'sii',
'year': year,
'season': season,
}

response = requests.post(url,form_data)
response.encoding = 'utf8'
response=response.text
# df = translate_dataFrame(response.text)

remove_one = response.split('<')
print('remove_one：',remove_one)

remove_two = remove_one[0].split('>')
print('remove_two：',remove_two)

# remove_two[1].replace(",", "")


remove_one： ['', 'HTML>\n', 'HEAD>\n', "script>\n\tvar url = window.location.href.replace('http://', 'https://');\n\twindow.location = url;\n", '/script>\n', 'TITLE>forward', '/TITLE>\n', '/HEAD>\n', '/HTML>']
remove_two： ['']


In [58]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def financial_statement(year, season, type='綜合損益彙總表'):
    if year >= 1000:
        year -= 1911
        
    if type == '綜合損益彙總表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb04'
    elif type == '資產負債彙總表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb05'
    elif type == '營益分析彙總表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb06'
    else:
        print('type does not match')
    r = requests.post(url, {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK':'sii',
        'year':'103',
        'season':'01',
    })
    
    r.encoding = 'utf8'
    dfs = pd.read_html(r.text)
    
    
    for i, df in enumerate(dfs):
        df.columns = df.iloc[0]
        dfs[i] = df.iloc[1:]
        
    df = pd.concat(dfs).applymap(lambda x: x if x != '--' else np.nan)
    df = df[df['公司代號'] != '公司代號']
    df = df[~df['公司代號'].isnull()]
    return df


df = financial_statement(107, 2, '營益分析彙總表')

df = df.drop(['合計：共 808 家'], axis=1)
df = df.set_index(['公司名稱'])
df = df.astype(float)

df['毛利率(%)(營業毛利)/(營業收入)'].hist(bins=range(-100,100))
df.loc['台積電']
df.loc[['台積電', '聯發科']]

#數值分析
#df.discribe()

plt.show()
df['毛利率(%)(營業毛利)/(營業收入)'].hist(bins=range(-100,100))

cond1 = df['毛利率(%)(營業毛利)/(營業收入)'].astype(float) > 20
cond2 = df['營業利益率(%)(營業利益)/(營業收入)'].astype(float) > 5
df[cond1 & cond2]
print(df)

ValueError: No tables found

In [11]:
def financial_statement(year, season):

    if year >= 1000:
        year -= 1911
        
    url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb06'
    form_data = {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK':'sii',
        'year': year,
        'season': season,
    }

    response = requests.post(url,form_data)
    response.encoding = 'utf8'
    
    df = translate_dataFrame(response.text)
    return df

stock = financial_statement(107,2)
stock = stock.astype(float)

IndexError: list index out of range

In [ ]:
plt.rcParams['axes.unicode_minus']=False
fig = plt.figure(figsize=(10, 6))
stock['毛利率(%)'].hist(bins=range(-100,100) , label="毛利率(%)")
plt.legend()

In [ ]:
cond1 = stock['毛利率(%)'] > 30
cond2 = stock['營業利益率(%)'] > 30
stock[cond1 & cond2]

In [ ]:
import requests
import time
import random
from bs4 import BeautifulSoup
from openpyxl import Workbook
from openpyxl import load_workbook
url = 'https://mops.twse.com.tw/mops/web/ajax_t100sb14'

url1 = 'https://mops.twse.com.tw/mops/web/t100sb14'

#尋找公司代號
def find_index(companys,as_companyno):      
    li_return = 0
    i = 0
    for company in companys:
        if company[1] == as_companyno:
           li_return = i
           break
        i = i + 1
    return li_return


#取得市場別id，其實就是上市及上櫃的值
def get_market():
    market_to_id = {}
    resp = requests.get(url1)
    soup = BeautifulSoup(resp.text, 'html5lib')
    selects = soup.find_all('select',{'name':'code'})
    options = soup.find('select',{'name':'TYPEK'}).find_all('option')
    i = 0
    for opt in options:
        market_to_id[opt.text.strip()] = opt['value']
    return market_to_id 

def gen_companylist():
    #==讀取上市及上櫃所有資料==========================================
    companys = list()
    title = list()
    wb1 = load_workbook('上市公司資料.xlsx')
    wb2 = load_workbook('上櫃公司資料.xlsx')
    ws1 = wb1.active
    ws2 = wb2.active
    
    i = 0
    for row in ws1.rows:
        company = list()
        if i == 0:
            for col in row:
                title.append(col.value)
            title.append('員工福利費用(仟元)')
            title.append('員工薪資費用(仟元)')            
            title.append('員工人數')
            title.append('平均員工福利費用(仟元/人)')  
            title.append('平均員工薪資費用(仟元/人)')
            title.append('每股盈餘(元/股)')  
            title.append('同產業平均員工福利費用(仟元/人)')
            title.append('同產業平均員工薪資費用(仟元/人)')              
            title.append('同產業平均每股盈餘(元/股)')     
            companys.append(title)  #加入上市公司資料    
        else:
            for col in row:
                company.append(col.value)
            companys.append(company)  #加入上市公司資料
        i = i + 1      
        

    i = 0
    for row in ws2.rows:
        company = list()
        if i == 0:
            #Do Nothing
            i == i
        else:
            for col in row:
                company.append(col.value)
            companys.append(company)  #加入上櫃公司資料
        i = i + 1 
    #===============================================================

    wb1.close()    #關閉工作簿
    wb2.close()    #關閉工作簿   
    return companys


#===========================主程式==================================
if __name__ == '__main__':   
    companys = gen_companylist() #抓取公司List
    #抓取網路相關資料
    market = get_market() #取得上市或是上櫃
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36'}
    
    for x in market:
        # 只抓股票名稱那欄  如果不給code數值，就是全部公司抓取，maket[x]代表上市或是上櫃公司
        d = {
            'encodeURIComponent':'1',
            'step':'1',
            'firstin':'1',
            'TYPEK': market[x],
            'RYEAR':'107',
            'code':''
            }
        r = requests.post(url, data=d,headers = headers)
        soup = BeautifulSoup(r.text,'html.parser')
        
        table = soup.find('table')
        rows = table.find_all('tr')
        #====Merger Two Lists==================================================
        i = 0
        for row in rows:
            if i <= 1:
                i = i
            else:
                company = list()
                for column in row.find_all('td'):
                    company.append(column.text.strip().replace(',',''))
                    #搜尋companys
                li_id = find_index(companys,company[1])
                j = 0
                for col in company:
                    if j >= 4: #從第五個欄位開始抓取
                       companys[li_id].append(col)
                    j = j + 1
                                
            i = i + 1

#======寫入Excel檔案======================
        wb3 = Workbook() #產生新的EXCEL
        ws3 = wb3.active
        for com in companys:
            ws3.append(list(com))           
        wb3.save('上市上櫃總名單.xlsx')                
        wb3.close()
        print('寫入資料' + market[x])
        print('休息10秒...')
        time.sleep(10)
        #休息10秒繼續抓，避免被網站認為是機器人攻擊

In [ ]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""
import requests
import time
import random
from bs4 import BeautifulSoup
from openpyxl import Workbook
from openpyxl import load_workbook

urls = list()
nonprofit = list()

#先抓出每一個非營利機構的超連結
for i in range(0,160):
    url = 'http://www.npo.org.tw/npolist_list.asp?nowpage=' + str(i) + '&npost=&lo=&keyword2='
    try:
        response = requests.get(url)
    except requests.exceptions.ConnectionError as e:
            print('Error', e.args)

    soup = BeautifulSoup(response.text,'html.parser')
    table = soup.find('table','common_table_01')
    tds = table.find_all('td','common_table_01_td_bg')
    for td in tds:
        urls.append(td.a['href'])
        

#再來抓出每一個營利機構的名稱與Email
for item in urls:
    url = 'http://www.npo.org.tw/' + item
    try:
        response = requests.get(url)
    except requests.exceptions.ConnectionError as e:
            print('Error', e.args)

    soup = BeautifulSoup(response.text,'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr')
    #print(rows[0].find_all('td')[0].text)
    #print(rows[7].find_all('td')[0].text)        
    nonprofit.append([rows[0].find_all('td')[0].text,rows[7].find_all('td')[0].text])
        
#======寫入Excel檔案======================
wb = Workbook() #產生新的EXCEL
ws = wb.active
ws.append(['機構','email'])   
for com in nonprofit:
    ws.append(list(com))           
wb.save('非營利機構總名單.xlsx')                
wb.close()

In [ ]:
#import相關套件
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
#輸入股票代碼及年份開始抓取資料

stock = input('輸入股票代碼')
year = input('輸入年份')
url = url= "http://mops.twse.com.tw/mops/web/ajax_t05st01?firstin=1&TYPEK=sii&co_id="+ str(stock) + "&year=" + str(year) + "&month=&b_date=&e_date="
page = requests.get(url)
page.encoding = 'utf-8'
soup = bs(page.text, 'lxml')
tb = soup.select('table')[1]
df = pd.read_html(tb.prettify('utf-8'), encoding= 'utf-8')
df[0]